## **Retrieval and Evaluation**

In [ ]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import os
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from joblib import load

# 1. Prepare data

## 1.1 Function for prepare data

In [ ]:
# paths to data
data_dir = '/kaggle/input/fashion-iq-dataset/fashionIQ_dataset'
image_dir = os.path.join(data_dir, 'images')
json_dir = os.path.join(data_dir, 'image_splits')

def read_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

#  load image (path of image) from file json (follow cattegory)
def load_image_list(category, split):
    json_path = os.path.join(json_dir, f'split.{category}.{split}.json')
    image_list = read_json(json_path)
    return [os.path.join(image_dir, image_name + '.jpg') for image_name in image_list]

# define transform for preprocess step
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# define class FashionIQDataset 
class FashionIQDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

## 1.2 Load features & Dataset

### a. Features

In [ ]:
X_train = np.load('/kaggle/input/features-and-labels-fashioniq/train_features.npy')
y_train = np.load('/kaggle/input/features-and-labels-fashioniq/train_labels.npy')

X_valid = np.load('/kaggle/input/features-and-labels-fashioniq/val_features.npy')
y_valid = np.load('/kaggle/input/features-and-labels-fashioniq/val_labels.npy')

X_test = np.load('/kaggle/input/features-and-labels-fashioniq/test_features.npy')
y_test = np.load('/kaggle/input/features-and-labels-fashioniq/test_labels.npy')

### b. Images

In [ ]:
# dress
train_images_dress = load_image_list('dress', 'train')
val_images_dress = load_image_list('dress', 'val')
test_images_dress = load_image_list('dress', 'test')

# category - shirt
train_images_shirt = load_image_list('shirt', 'train')
val_images_shirt = load_image_list('shirt', 'val')
test_images_shirt = load_image_list('shirt', 'test')

# category - top&tee
train_images_toptee = load_image_list('toptee', 'train')
val_images_toptee = load_image_list('toptee', 'val')
test_images_toptee = load_image_list('toptee', 'test')

dress = train_images_dress + val_images_dress + test_images_dress
shirt = train_images_shirt + val_images_shirt + test_images_shirt
toptee = train_images_toptee + val_images_toptee + test_images_toptee

dress_labels = [0] * len(dress)
shirt_labels = [1] * len(shirt)
toptee_labels = [2] *len(toptee)

In [ ]:
train_images = train_images_dress + train_images_shirt + train_images_toptee
val_images = val_images_dress + val_images_shirt + val_images_toptee
test_images = test_images_dress + test_images_shirt + test_images_toptee

## 1.2. Load Model

### a. SVM

In [ ]:
# Load the saved model (from pretraining)
svm_model = load('/kaggle/input/svm-model/svm_model.joblib')

### b. ResNet50

In [ ]:
# nếu sử dụng gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = '/kaggle/input/resnet50-weight-fashion/best_model.pth'
num_classes = 3

# Sử dụng mô hình resnet50 với trọng số đã huấn luyện
model = models.resnet50(weights = None)  
model.fc = nn.Linear(model.fc.in_features, num_classes)

# load 
model.load_state_dict(torch.load(model_dir))

model.to(device)  
# Chuyển mô hình về chế độ đánh giá (evaluation mode)
model.eval()

# 2. Retrieval

## 2.1 Accuracy functions

In [ ]:
def calculate_precision_at_k(query_label, similar_labels, k):
    # Tính precision tại top-k
    relevant_items = np.array(similar_labels)[:k] == query_label
    precision_at_k = np.sum(relevant_items) / k
    return precision_at_k

def calculate_average_precision(query_label, similar_labels):
    relevant_items = np.array(similar_labels) == query_label
    num_relevant_items = np.sum(relevant_items)
    
    if num_relevant_items == 0:
        return 0.0
    
    # Tính AP bằng cách tính Precision tại mỗi k có kết quả đúng
    precision_sum = 0.0
    correct_retrieved = 0
    
    for k in range(1, len(similar_labels) + 1):
        if relevant_items[k - 1]:
            correct_retrieved += 1
            precision_at_k = correct_retrieved / k
            precision_sum += precision_at_k
    
    # Average Precision là trung bình của Precision@k tại các vị trí đúng
    return precision_sum / num_relevant_items

def calculate_map(query_label, similar_labels):    
    return calculate_average_precision(query_label, similar_labels)

## 2.2 Retrival functions

In [ ]:
## Hàm đánh giá kết quả truy vấn trên toàn bộ tập test với k = {10, 100, 1000, 10000}
def retrive_impove(X_test, y_test, top_k_values):
    results = {}
    
    for top_k in top_k_values:
        accuracies = []
        times = []
        all_map = []

        for i in range(len(X_test)):
            # Chọn ảnh từ tập test
            input_feature = X_test[i]
            input_label = y_test[i]

            # Tính toán độ tương tự
            start_time = time.time()

            # phân loại
            label_predict = svm_model.predict([input_feature])
            if(label_predict == 0):
                train_features = np.load('/kaggle/working/dress_features.npy')
                train_labels = 0
                train_images = dress
            elif(label_predict == 1):
                train_features = np.load('/kaggle/working/shirt_features.npy')
                train_labels = 1
                train_images = shirt
            else: 
                train_features = np.load('/kaggle/working/toptee_features.npy')
                train_labels = 2
                train_images = toptee
                
            similarities = cosine_similarity([input_feature], train_features)[0]
            top_k_indices = np.argsort(similarities)[-top_k:][::-1]
            end_time = time.time()
            
            # Tính thời gian truy vấn
            query_time = end_time - start_time
            times.append(query_time)

            # Tính độ chính xác
            correct = accuracy_score([label_predict], [input_label])
            if(correct == 1.0):
                i_mAP = 1
            else: i_mAP = 0
            
            accuracies.append(correct)
            all_map.append(i_mAP)
            
#             print(i, label_predict, input_label, i_mAP, correct)
        
        # Tính độ chính xác trung bình và thời gian truy vấn trung bình
        accuracy_at_k = np.mean(accuracies)
        map_at_k = np.mean(all_map)
        mean_time = np.mean(times)
        
        results[top_k] = {
            'mean_accuracy': accuracy_at_k,
            'mean_time': mean_time
        }

        print(f"Top-{top_k}:")
        print(f"  mAP: {map_at_k:.3f}")
        print(f"  Mean Accuracy: {accuracy_at_k:.3f}")
        print(f"  Mean Query Time: {mean_time:.3f} seconds")
    
    return results



# Các giá trị top-k cần kiểm tra
top_k_values = [10, 100, 1000, 10000]
# Đánh giá hiệu suất truy vấn
results = retrive_impove(X_test, y_test, top_k_values)


## ResNet50

In [ ]:
# Hàm tính accuracy
def calculate_precision_at_k(query_label, similar_labels, k):
    # Tính precision tại top-k
    relevant_items = np.array(similar_labels)[:k] == query_label
    precision_at_k = np.sum(relevant_items) / k
    return precision_at_k

def calculate_average_precision(query_label, similar_labels):
    relevant_items = np.array(similar_labels) == query_label
    num_relevant_items = np.sum(relevant_items)
    
    if num_relevant_items == 0:
        return 0.0
    
    # Tính AP bằng cách tính Precision tại mỗi k có kết quả đúng
    precision_sum = 0.0
    correct_retrieved = 0
    
    for k in range(1, len(similar_labels) + 1):
        if relevant_items[k - 1]:
            correct_retrieved += 1
            precision_at_k = correct_retrieved / k
            precision_sum += precision_at_k
    
    # Average Precision là trung bình của Precision@k tại các vị trí đúng
    return precision_sum / num_relevant_items

def calculate_map(query_label, similar_labels):    
    return calculate_average_precision(query_label, similar_labels)

def calculate_accuracy(query_label, similar_labels):
    num_correct = np.sum(np.array(similar_labels) == query_label)
    num_total = len(similar_labels)
    accuracy = num_correct / num_total
    return accuracy

def evaluate_query_performance(X_test, y_test, top_k_values):
    results = {}
    all_features = np.load("/kaggle/input/features/all_features.npy")
    all_labels = np.load("/kaggle/input/features/all_labels.npy")
    
    for top_k in top_k_values:
        accuracies = []
        times = []
        all_map = []

        for i in range(len(X_test)):
            # Chọn ảnh từ tập test
            input_feature = X_test[i]
            input_label = y_test[i]

            # Tính toán độ tương tự
            start_time = time.time()
                
            similarities = cosine_similarity([input_feature], all_features)[0]
            top_k_indices = np.argsort(similarities)[-top_k:][::-1]
            top_k_labels = np.array(all_labels)[top_k_indices]
            end_time = time.time()
            
            # Tính thời gian truy vấn
            query_time = end_time - start_time
            times.append(query_time)

            # Tính độ chính xác
            i_map = calculate_map(input_label, top_k_labels)
            accuracy = calculate_accuracy(input_label, top_k_labels)
            
            accuracies.append(int(accuracy))
            all_map.append(float(i_map))
        
        # Tính độ chính xác trung bình và thời gian truy vấn trung bình
        accuracy_at_k = np.mean(accuracies)
        map_at_k = np.mean(all_map)
        mean_time = np.mean(times)
        
        results[top_k] = {
            'mean_accuracy': accuracy_at_k,
            'mean_time': mean_time
        }

        print(f"Top-{top_k}:")
        print(f"  Mean mAP: {map_at_k:.3f}")
        print(f"  Mean Accuracy: {accuracy_at_k:.3f}")
        print(f"  Mean Query Time: {mean_time:.3f} seconds")
    
    return results

# Các giá trị top-k cần kiểm tra
top_k_values = [10, 100, 1000, 10000]

# Đánh giá hiệu suất truy vấn
results = evaluate_query_performance(X_test, y_test, top_k_values)